In [29]:
from pyspark import SparkConf, SparkContext
from pyspark.sql.functions import *
sc.stop()
from pyspark.sql import SQLContext
sc =SparkContext()
sqlContext = SQLContext(sc)
import pyspark.sql.functions as func
import sys
from pyspark.sql.functions import countDistinct

In [30]:
#reading in sticker type data
main_data = sc.textFile("gs://ds-url-catag/Emails_verifalia/email_100_b.csv")
header = main_data.take(1)[0]
filter_header = main_data.filter(lambda line: line != header) \
              .map(lambda line: line.split(","))  \
              .map(lambda line: (line[0],line[1])).toDF(['email','language'])



In [20]:
filter_header.count()

9407056

In [21]:
filter_header.distinct().count()

9405982

In [31]:
distinct_emails = filter_header.distinct()

In [17]:
# #testing for sample
# sample_size_test = distinct_emails.limit(10)
# sample_size_test.write.mode('overwrite').parquet("gs://ds-url-catag/Emails_verifalia/testing/")
# read_again = sqlContext.read.parquet("gs://ds-url-catag/Emails_verifalia/testing/")
# read_again.show()

In [32]:
from pyspark.sql.functions import col  
from pyspark.sql.window import Window


w = Window().orderBy('email')
distinct_emails= distinct_emails.withColumn("rowNum",func.row_number().over(w))

In [ ]:
distinct_emails.write.mode('overwrite').parquet("gs://ds-url-catag/Emails_verifalia/testing/")

In [10]:
new_data = sqlContext.read.parquet("gs://ds-url-catag/Emails_verifalia/testing/")

In [ ]:
new_data.show(100)

In [ ]:
new_data= new_data.withColumn("mod_num",lit(160))
new_data= new_data.withColumn("every_160",col('rowNum')%col('mod_num'))


In [ ]:
Sample = new_data.where(col('mod_num') == 0)

In [ ]:
Sample.count()

In [ ]:
Sample.select('email').limit(50000).write.text("gs://ds-url-catag/Emails_verifalia/sample_email")